In [1]:
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [2]:
# Step 1: Load and preprocess data
def load_cifar10_data(data_dir):
    X = []
    Y = []
    for i in range(1, 6):
        with open(os.path.join(data_dir, f'data_batch_{i}'), 'rb') as file:
            batch = pickle.load(file, encoding='latin1')
            X.append(batch['data'])
            Y.extend(batch['labels'])
    X = np.concatenate(X).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    X_test, Y_test = None, None
    with open(os.path.join(data_dir, 'test_batch'), 'rb') as file:
        test_batch = pickle.load(file, encoding='latin1')
        X_test = test_batch['data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        Y_test = test_batch['labels']
    X = np.vstack((X, X_test))
    Y.extend(Y_test)
    Y = np.array(Y)
    return X, Y

In [3]:
# Step 2: Build the MLP model
def build_mlp_model(input_shape, num_classes, learning_rate, num_units, dropout_rate):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(num_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [4]:
# Step 3: Define cross-validation function
def cross_validate_and_select_best_params(model, X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=5, batch_size=128, verbose=0)
        scores = model.evaluate(X_test, y_test, verbose=0)
        accuracies.append(scores[1])
    return np.mean(accuracies)

In [5]:
# Load CIFAR-10 data
data_dir = 'cifar-10-batches-py'
X_combined, y_combined = load_cifar10_data(data_dir)
X_combined = X_combined.astype('float32') / 255.0
y_combined = to_categorical(y_combined, 10)

In [8]:
# Step 4: Iterate over parameter combinations
learning_rates = [0.001, 0.0005, 0.0015, 0.01, 0.005]
unit_sizes = [8, 16, 32,64, 128]
dropout_rates = [0, 0.1, 0.3, 0.5, 0.75]
highest_accuracy = 0
highest_config = []

for learning_rate in learning_rates:
    for num_units in unit_sizes:
        for dropout_rate in dropout_rates:
            model = build_mlp_model((32, 32, 3), 10, learning_rate, num_units, dropout_rate)
            avg_accuracy = cross_validate_and_select_best_params(model, X_combined, y_combined)
            print(f"Config: LR={learning_rate}, Units={num_units}, Dropout={dropout_rate} | Accuracy={avg_accuracy:.4f}")
            if avg_accuracy > highest_accuracy:
                highest_accuracy = avg_accuracy
                highest_config = [learning_rate, num_units, dropout_rate]
                print("New highest found:", highest_config, "with accuracy:", highest_accuracy)

print("Highest configuration:", highest_config)
print("Highest accuracy:", highest_accuracy)

Config: LR=0.001, Units=8, Dropout=0 | Accuracy=0.0954
New highest found: [0.001, 8, 0] with accuracy: 0.09543333202600479
Config: LR=0.001, Units=8, Dropout=0.1 | Accuracy=0.0956
New highest found: [0.001, 8, 0.1] with accuracy: 0.09563333243131637
Config: LR=0.001, Units=8, Dropout=0.3 | Accuracy=0.0956
Config: LR=0.001, Units=8, Dropout=0.5 | Accuracy=0.0953
Config: LR=0.001, Units=8, Dropout=0.75 | Accuracy=0.0953
Config: LR=0.001, Units=16, Dropout=0 | Accuracy=0.3698
New highest found: [0.001, 16, 0] with accuracy: 0.36978332996368407
Config: LR=0.001, Units=16, Dropout=0.1 | Accuracy=0.3237
Config: LR=0.001, Units=16, Dropout=0.3 | Accuracy=0.1869
Config: LR=0.001, Units=16, Dropout=0.5 | Accuracy=0.0953
Config: LR=0.001, Units=16, Dropout=0.75 | Accuracy=0.0959
Config: LR=0.001, Units=32, Dropout=0 | Accuracy=0.3720
New highest found: [0.001, 32, 0] with accuracy: 0.3720333337783813
Config: LR=0.001, Units=32, Dropout=0.1 | Accuracy=0.3051
Config: LR=0.001, Units=32, Dropout=0.

In [9]:
# Step 4: Iterate over parameter combinations
learning_rates = [0.001, 0.0005, 0.0015, 0.01, 0.005]
unit_sizes = [256, 512]
dropout_rates = [0, 0.1, 0.3, 0.5, 0.75]
highest_accuracy = 0
highest_config = []

for learning_rate in learning_rates:
    for num_units in unit_sizes:
        for dropout_rate in dropout_rates:
            model = build_mlp_model((32, 32, 3), 10, learning_rate, num_units, dropout_rate)
            avg_accuracy = cross_validate_and_select_best_params(model, X_combined, y_combined)
            print(f"Config: LR={learning_rate}, Units={num_units}, Dropout={dropout_rate} | Accuracy={avg_accuracy:.4f}")
            if avg_accuracy > highest_accuracy:
                highest_accuracy = avg_accuracy
                highest_config = [learning_rate, num_units, dropout_rate]
                print("New highest found:", highest_config, "with accuracy:", highest_accuracy)

print("Highest configuration:", highest_config)
print("Highest accuracy:", highest_accuracy)

Config: LR=0.001, Units=256, Dropout=0 | Accuracy=0.5147
New highest found: [0.001, 256, 0] with accuracy: 0.514683336019516
Config: LR=0.001, Units=256, Dropout=0.1 | Accuracy=0.4824
Config: LR=0.001, Units=256, Dropout=0.3 | Accuracy=0.4097
Config: LR=0.001, Units=256, Dropout=0.5 | Accuracy=0.2617
Config: LR=0.001, Units=256, Dropout=0.75 | Accuracy=0.0978
Config: LR=0.001, Units=512, Dropout=0 | Accuracy=0.5284
New highest found: [0.001, 512, 0] with accuracy: 0.5284333229064941
Config: LR=0.001, Units=512, Dropout=0.1 | Accuracy=0.4959
Config: LR=0.001, Units=512, Dropout=0.3 | Accuracy=0.4297
Config: LR=0.001, Units=512, Dropout=0.5 | Accuracy=0.3306
Config: LR=0.001, Units=512, Dropout=0.75 | Accuracy=0.0966
Config: LR=0.0005, Units=256, Dropout=0 | Accuracy=0.5260
Config: LR=0.0005, Units=256, Dropout=0.1 | Accuracy=0.5065
Config: LR=0.0005, Units=256, Dropout=0.3 | Accuracy=0.4452
Config: LR=0.0005, Units=256, Dropout=0.5 | Accuracy=0.3339
Config: LR=0.0005, Units=256, Dropout

In [12]:
# Step 4: Iterate over parameter combinations
learning_rates = [0.0005]
unit_sizes = [1024]
dropout_rates = [0]
highest_accuracy = 0
highest_config = []

for learning_rate in learning_rates:
    for num_units in unit_sizes:
        for dropout_rate in dropout_rates:
            model = build_mlp_model((32, 32, 3), 10, learning_rate, num_units, dropout_rate)
            avg_accuracy = cross_validate_and_select_best_params(model, X_combined, y_combined)
            print(f"Config: LR={learning_rate}, Units={num_units}, Dropout={dropout_rate} | Accuracy={avg_accuracy:.4f}")
            if avg_accuracy > highest_accuracy:
                highest_accuracy = avg_accuracy
                highest_config = [learning_rate, num_units, dropout_rate]
                print("New highest found:", highest_config, "with accuracy:", highest_accuracy)

print("Highest configuration:", highest_config)
print("Highest accuracy:", highest_accuracy)

Config: LR=0.0005, Units=1024, Dropout=0 | Accuracy=0.5608
New highest found: [0.0005, 1024, 0] with accuracy: 0.560783326625824
Highest configuration: [0.0005, 1024, 0]
Highest accuracy: 0.560783326625824


In [17]:
# Step 4: Iterate over parameter combinations
learning_rates = [0.0005]
unit_sizes = [2048]
dropout_rates = [0]
highest_accuracy = 0
highest_config = []

for learning_rate in learning_rates:
    for num_units in unit_sizes:
        for dropout_rate in dropout_rates:
            model = build_mlp_model((32, 32, 3), 10, learning_rate, num_units, dropout_rate)
            avg_accuracy = cross_validate_and_select_best_params(model, X_combined, y_combined)
            print(f"Config: LR={learning_rate}, Units={num_units}, Dropout={dropout_rate} | Accuracy={avg_accuracy:.4f}")
            if avg_accuracy > highest_accuracy:
                highest_accuracy = avg_accuracy
                highest_config = [learning_rate, num_units, dropout_rate]
                print("New highest found:", highest_config, "with accuracy:", highest_accuracy)

print("Highest configuration:", highest_config)
print("Highest accuracy:", highest_accuracy)

Config: LR=0.0005, Units=2048, Dropout=0 | Accuracy=0.5691
New highest found: [0.0005, 2048, 0] with accuracy: 0.5690999984741211
Highest configuration: [0.0005, 2048, 0]
Highest accuracy: 0.5690999984741211


In [25]:
# Step 3: Define cross-validation function
def cross_validate_and_select_best_params(model, X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=0)
        scores = model.evaluate(X_test, y_test, verbose=0)
        accuracies.append(scores[1])
    return np.mean(accuracies)

In [27]:
# Step 4: Iterate over parameter combinations
learning_rates = [0.0005]
unit_sizes = [2048]
dropout_rates = [0]
highest_accuracy = 0
highest_config = []

for learning_rate in learning_rates:
    for num_units in unit_sizes:
        for dropout_rate in dropout_rates:
            model = build_mlp_model((32, 32, 3), 10, learning_rate, num_units, dropout_rate)
            avg_accuracy = cross_validate_and_select_best_params(model, X_combined, y_combined)
            print(f"Config: LR={learning_rate}, Units={num_units}, Dropout={dropout_rate} | Accuracy={avg_accuracy:.4f}")
            if avg_accuracy > highest_accuracy:
                highest_accuracy = avg_accuracy
                highest_config = [learning_rate, num_units, dropout_rate]
                print("New highest found:", highest_config, "with accuracy:", highest_accuracy)

print("Highest configuration:", highest_config)
print("Highest accuracy:", highest_accuracy)

Config: LR=0.0005, Units=2048, Dropout=0 | Accuracy=0.6490
New highest found: [0.0005, 2048, 0] with accuracy: 0.6490333437919616
Highest configuration: [0.0005, 2048, 0]
Highest accuracy: 0.6490333437919616
